# Computational Theory Assement Problems

## Problem 1: Binary Words and Operations

In [4]:
import numpy as np

In [5]:
#Global SHA-256 constants
sha256_constants = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]
#global initial hash values for SHA-256 (current values in problem 4)
sha256_initial_hash_values = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
    ]

In [6]:
#1.
# ^ = bitwise XOR
def Parity(x,y,z):
    """
    Performs bitwise XOR(exclusive or) operation on each bit position of the 3 values
    x == 0 y == 0 z == 0 result == 0
    x == 1 y == 0 z == 0 result == 1 (returns 1 when odd number of 1's)
    x == 1 y == 1 z == 0 result == 0 (returns 0 when even number of 1's)
    x == 1 y == 1 z == 1 result == 1
    Arg:

    Parameters: 
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned  integer

    Returns:
    np.uint32
        value made up of results for each bit position of the 3 input XOR
    """
    return np.uint32(x) ^ np.uint32(y) ^ np.uint32(z)

#Test 
#input
x = 0b1101  # 13 
y = 0b0111  # 7 
z = 0b0000  # 0 
#expected 
expected = 0b1010 # 10
#actual
actual = Parity(x,y,z)
print(f"Input: x={x} ({bin(x)}), y={y} ({bin(y)}), z={z} ({bin(z)})")
print(f"Result: {actual} ({bin(actual)})")
print(f"Expected: {expected} ({bin(expected)})")
print(f"Test Passed: {actual == expected}")


Input: x=13 (0b1101), y=7 (0b111), z=0 (0b0)
Result: 10 (0b1010)
Expected: 10 (0b1010)
Test Passed: True


In [7]:
#2.
# & = bitwise and 
# ~ = not operator (flips all bits i.e 0x00000000 -> 0xFFFFFFFF)
def Ch(x,y,z):
    """
    returns new value built from the taken bits of y and z based on the state of x

    For each bit position in x check if the bit is 1.
    if it is, take the corresponding bit in y 
    if not, take the corresponding bit in z

    Parameters
    ----------
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned integer

    Returns
    ---------
    np.unit32 
        value made from z and y based on the state of x
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (~x & z)

# Test 
x = 0b1100  # 12 
y = 0b1010  # 10 
z = 0b0110  # 6 

#expected result
expected = 0b1010  # 10
#actual result
actual = Ch(x, y, z)
print(f"Input: x={x} ({bin(x)}), y={y} ({bin(y)}), z={z} ({bin(z)})")
print(f"Result: {actual} ({bin(actual)})")
print(f"Expected: {expected} ({bin(expected)})")
print(f"Test Passed: {actual == expected}")


Input: x=12 (0b1100), y=10 (0b1010), z=6 (0b110)
Result: 10 (0b1010)
Expected: 10 (0b1010)
Test Passed: True


In [8]:
#3.
def Maj(x,y,z):
    """ 
    Returns 32 bit integer bitwise majority

    For each bit position, returns 1 if two or more of the three inputs 
    have that bit set to 1 else returns 0.

    Parameters
    ---------
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned integer

    Returns
    ------- 
    np.uint32
        value made up of the majority of each bit position of the 3 values
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    #For each bit position perform a bitwise AND between each of the 3 values
    #which returns 1 if both bits are 1 or 0 in all other cases, then perform a 3 input XOR
    #using the values returned from the AND comparisons 

    return (x & y) ^ (x & z) ^ (y & z)

# Test 
x = 0b1100  # 12
y = 0b1010  # 10
z = 0b1001  # 9

expected = 0b1000  # 8
actual = Maj(x, y, z)
print(f"Input: x={x} ({bin(x)}), y={y} ({bin(y)}), z={z} ({bin(z)})")
print(f"Result: {actual} ({bin(actual)})")
print(f"Expected: {expected} ({bin(expected)})")
print(f"Test Passed: {actual == expected}")

Input: x=12 (0b1100), y=10 (0b1010), z=9 (0b1001)
Result: 8 (0b1000)
Expected: 8 (0b1000)
Test Passed: True


In [9]:
#4.
# | = bitwise inclusive or
# W = number of bits 
# >> = shifts each bit to the right bits on the right fall off and are replaced on the left by 0's
# i.e n = 3   11000-->00011
# << = shifts each bit to the left bits on the left fall off and are replaced on the right by 0's
#i.e n = 3 11001->01000
def ROTR(x,n):
    """ 
    Executes  rotate right (circular right shift) operation on 32 bit unsigned integer
    Rotates all bits in X to the right by n.Any bits that fall off are wrapped to the left
    end

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 
    N: int 
    number of bits to rotate x by 0-31

    Returns
    ------- 
    np.uint32
        Resulting rotation of x by n bit positions
        
    """
    x = np.uint32(x)
    n = np.uint32(n)
    w = 32
    #shifts each bit in x to the right by n positions
    #left shift the bits of x by the bit size of x-n(32-n) 
    #combine both shifted values using bitwise OR to return the rotated result
    #& 0xFFFFFFFF Ensures results remain within 32bits by zeroing any bits over 32
    # https://stackoverflow.com/questions/10493411/what-is-bit-masking  I used this to understand how bitmasking works and why it might be nessarcary
    return ((x >> n) | (x << (w - n))) & 0xFFFFFFFF 

#test 
#inputs
x = 0b11000000000000000000000000000000  
n = 3 # rotate by 3
#outputs
expected = 0b00011000000000000000000000000000  # Rotated right by 3
actual = ROTR(x, n)
print(f"Input: x={x} ({bin(x)}), n={n}")
print(f"Result: {actual} (0b{actual:032b})")
print(f"Expected: {expected} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=3221225472 (0b11000000000000000000000000000000), n=3
Result: 402653184 (0b00011000000000000000000000000000)
Expected: 402653184 (0b00011000000000000000000000000000)
Test Passed: True


In [10]:
def SHR(x,n):
    """Executes the right shift operation on a 32-bit integer.
    shifts each bit position of x by n positions.Any bit that falls off is replaced by a 0 from the left end
    
    Parameters
    ---------
    X: int 
    32 bit unsigned integer 
    N: int 
    number of bits to shift x by 0-31

    Returns
    ------- 
    np.uint32
        Resulting shift of x by n bit positions
    
    """
    x = np.uint32(x)
    n = np.uint32(n)
    #shift bit positions of x by n positions and return the shifted 32 bit integer
    return np.uint32( x >> n)

# Test
# inputs
x = 0b11000000000000000000000000000000  
n = 3# shift by 3
# outputs
expected = 0b00011000000000000000000000000000  # Shifted right by 3
actual = SHR(x, n)


print(f"Input: x={x} (0b{x:032b}), n={n}")
print(f"Result: {actual} (0b{actual:032b})")
print(f"Expected: {expected} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=3221225472 (0b11000000000000000000000000000000), n=3
Result: 402653184 (0b00011000000000000000000000000000)
Expected: 402653184 (0b00011000000000000000000000000000)
Test Passed: True


In [11]:
def Sigma0(x):
    """
    SHA-256 Sigma0 (Σ₀)

    Performs 3 input XOR using 3 seperate right rotated values of x,
      each with differing rotation sizes(2, 13, and 22 bit positions)

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 

    Returns
    -------
    np.uint32
        Result of the 3 input XOR on the rotated values
    """
    x = np.uint32(x)
    return ROTR(x,2) ^ ROTR(x,13) ^ ROTR(x,22)

#test 
#input
x = 0b10101010101010101010101010101010  
#outputs
expected = 0b01010101010101010101010101010101
actual = Sigma0(x)

print(f"Input: x={x} (0b{x:032b})")
print(f"Result: {actual} (0b{actual:032b})")
print(f"Expected: {expected} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=2863311530 (0b10101010101010101010101010101010)
Result: 1431655765 (0b01010101010101010101010101010101)
Expected: 1431655765 (0b01010101010101010101010101010101)
Test Passed: True


In [12]:
#5.
def Sigma1(x):
    """
    SHA-256 Sigma1 (Σ₁) function

    Performs 3 input XOR using 3 seperate right rotated values of x,
      each with differing rotation sizes(6, 11, and 25 bit positions) 
      

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 3 rotated values 
    """
    x = np.uint32(x)
    return ROTR(x,6) ^ ROTR(x,11) ^ ROTR(x,25)

#test
#input
x = 0b10000000000000000000000000000000  
#outputs
expected = 0b00000010000100000000000001000000 
actual = Sigma1(x)

print(f"Input: x=0x{x:08X} (0b{x:032b})")
print(f"Result: 0x{actual:08X} (0b{actual:032b})")
print(f"Expected: 0x{expected:08X} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=0x80000000 (0b10000000000000000000000000000000)
Result: 0x02100040 (0b00000010000100000000000001000000)
Expected: 0x02100040 (0b00000010000100000000000001000000)
Test Passed: True


In [13]:
#6.
def Sigma0_2(x):
    """
    SHA-256 sigma0 (σ₀)

    Performs 3 input XOR using 2 seperate right rotated values of x,
      each with differing rotation sizes(7, and 18 bit positions) 
      and one right shifted value of x shifted by 3 bit positions

    Parameters
    ---------
    X: int 
    32 bit integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 2 rotated values of x and 1 right shifted value of x
    """
    x = np.uint32(x)
    return ROTR(x,7) ^ ROTR(x,18) ^ SHR(x,3)
#test
#input
x = 0b10000000000000000000000000000000 
#outputs
expected = 0b00010001000000000010000000000000 
actual = Sigma0_2(x)


print(f"Input: x=0x{x:08X} (0b{x:032b})")
print(f"Result: 0x{actual:08X} (0b{actual:032b})")
print(f"Expected: 0x{expected:08X} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x=0x80000000 (0b10000000000000000000000000000000)
Result: 0x11002000 (0b00010001000000000010000000000000)
Expected: 0x11002000 (0b00010001000000000010000000000000)
Test Passed: True


In [14]:
#7.
def Sigma1_2(x):
    """
    SHA-256 sigma1 (σ₁)
    Performs 3 input XOR using 2 seperate right rotated values of x
      each with differing rotation sizes(17, and 19 bit positions) 
      and one right shifted value of x shifted by 10 bit positions
      
    Parameters
    ---------
    X: int 
    32 bit integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 2 rotated values of x and 1 right shifted value of x
    """
    x = np.uint32(x)
    return ROTR(x,17) ^ ROTR(x,19) ^ SHR(x,10)
#test
#input
x = 0b10000000000000000000000000000000  
#outputs
expected = 0b00000000001000000101000000000000 
actual = Sigma1_2(x)

print(f"Input: x= {x} (0b{x:032b})")
print(f"Result: {actual} (0b{actual:032b})")
print(f"Expected: {expected} (0b{expected:032b})")
print(f"Test Passed: {actual == expected}")

Input: x= 2147483648 (0b10000000000000000000000000000000)
Result: 2117632 (0b00000000001000000101000000000000)
Expected: 2117632 (0b00000000001000000101000000000000)
Test Passed: True


## Problem 2: Fractional Parts of Cube Roots

In [15]:
#Part 1
"""
Generates the first n prime numbers and returns them in an list

Parameters
----------
n: int
    number of prime numbers to generate

Returns
-------
list
    list of the first n prime numbers
    
References: https://www.geeksforgeeks.org/dsa/generate-and-print-first-n-prime-numbers/ broke down how to find the first N prime numbers
"""
def primes(n):
    if n <= 0:
        return []
    #current number of found primes   
    x = 0
    #current number to check for prime
    y = 2
    #array to store prime numbers in
    primes = []
    #prime flagger
    flag = False
    #loop until n primes have been found
    while(x < n):
        #assume the number is prime
        flag = True
        #loop over and check if y is divisible by any number from 2 to sqrt(y)
        #if y is not prime it can be factored into 2 factors a and b
        #y = a*b and at least one of those factors must be less than or equal to sqrt(y)
        for j in range(2, int(np.floor(np.sqrt(y))) + 1):
            #check if y is divisible by j
            if(y % j == 0):
                #if it is then y is not prime
                flag = False
                break
        #if the number is still flagged as prime as it passed all divisibility tests
        if(flag):
            #increment count of found primes 
            #and add the prime to the list
            x+=1
            primes.append(y)
        #increment number to check for prime
        y+=1    
    return primes




In [ ]:
#Part 2-4
def generate_sha256_constants(n):
    """
    Generate first n prime numbers then Calculates the cube roots.
    Extracts the fractional part of a number by subtracting the floored value from the original number.
    Shift the fractional part by 32 binary places(2^32) and convert to integer to  remove any remaining decimal values.
    convert the resulting integers to hex and save as an array.
    Parameters
    ----------
    cubeRoots_primes64: float
        number to extract fractional part from

    Returns
    -------
    list
        fractional part of cubeRoots_primes64 shifted by 32 bit positions and converted to hex
    """
    # Step 1: Generate first n primes
    primes_list = primes(n)
    
    # Step 2: Calculate cube roots
    cube_roots = np.cbrt(primes_list)
    hex_roots = []
    #loop over each cube root of the first 64 primes
    for root in cube_roots:
        #extract fractional part by subtracting the value before the decimal from the original value
        fractional_part = root - np.floor(root)
        #shift the fractional part by 32 bit positions
        shifted_fractional = fractional_part * (2**32)
        #convert to integer to remove any remaining decimal values
        integer_fractional = int(shifted_fractional)
        #Part 4
        #convert to hex and append to array
        hex_roots.append(hex(integer_fractional))
    return hex_roots
hex_values = generate_sha256_constants(64)
print(hex_values)

['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5', '0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174', '0xe49b69c1', '0xefbe4786', '0xfc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da', '0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x6ca6351', '0x14292967', '0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85', '0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070', '0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3', '0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f2']


In [17]:
#part 5
def  SHScomparison(generated_constants):
    """
    Compares the generated SHA-512 constants with the official SHA-512 constants
    
    Parameters
    ----------
    generated_constants : list
        The list of generated SHA-512 constants to compare
    Returns
    -------
    bool
        True if the generated constants match the official constants, False otherwise
    """
    #SHA-256 constants
    
    #compare the generated constants with the official SHA-256 constants
    AllMatch = True
    for i in range(64):
        if generated_constants[i] == hex(sha256_constants[i]):
            print(f"[{i}] matches")
        else:
            print(f"[{i}] mismatch")
            AllMatch = False
    return print("AllMatch" if AllMatch else "Not all matched")

SHScomparison(hex_values)
    



[0] matches
[1] matches
[2] matches
[3] matches
[4] matches
[5] matches
[6] matches
[7] matches
[8] matches
[9] matches
[10] matches
[11] matches
[12] matches
[13] matches
[14] matches
[15] matches
[16] matches
[17] matches
[18] matches
[19] matches
[20] matches
[21] matches
[22] matches
[23] matches
[24] matches
[25] matches
[26] matches
[27] matches
[28] matches
[29] matches
[30] matches
[31] matches
[32] matches
[33] matches
[34] matches
[35] matches
[36] matches
[37] matches
[38] matches
[39] matches
[40] matches
[41] matches
[42] matches
[43] matches
[44] matches
[45] matches
[46] matches
[47] matches
[48] matches
[49] matches
[50] matches
[51] matches
[52] matches
[53] matches
[54] matches
[55] matches
[56] matches
[57] matches
[58] matches
[59] matches
[60] matches
[61] matches
[62] matches
[63] matches
AllMatch


## Problem 3: Padding

In [18]:
def block_parse(msg):
    """ Yields 512 bit (64 byte) blocks from the input message after padding according to SHA-256 specifications
    Parameters
    ----------
    msg: bytes
        input message to be padded and split into blocks            
    Yields
    -------
    bytes
        64 byte (512 bit) blocks of the padded message
    """
    #https://realpython.com/introduction-to-python-generators/
    
    #get msg length in bits and save before padding
    msg_len= len(msg) * 8
    
    #convert msg to bytearray to allow appending of bytes
    padded = bytearray(msg)
    
    #append the bit '1' to the end as the secure hashing standard specifies
    #to mark the end of the original message and start of padding
    padded.append(0x80)

    #get current length of the padded message
    current_len = len(padded)
    
    #check how far into current 64 byte(512 bit) block we are
    #check how many more bytes are needed to reach 56 bytes(448 bits)(usable number of bits in block after padding)
    #wrap to next block if already at or past 56 bytes
    padding_needed = (56 - current_len % 64) % 64
    #reserve 8 bytes(64 bits) for original message length
    padded.extend(b'\x00' * padding_needed)
    #append original message length as a 64 bit big-endian integer(nost significant byte first)
    padded.extend(msg_len.to_bytes(8, byteorder='big'))
    
    # Verify total length is multiple of 64 bytes (512 bits)
    assert len(padded) % 64 == 0, "padded message must be multiple of 512 bits"
    
    # Yield blocks of 64 bytes (512 bits)
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])

## Problem 4: Hashes

In [19]:
def hash(current, block):
    """
    Calculates the next hash value given the current hash value and the next message block
    according to  secure hash standard 6.2.2 sha-256 .
    
    Parameters
    ----------
        current: tuple of 8 32-bit integers  - current hash values
        block: message block of exactly 64 bytes (512 bits) - (output from block_parse)
    
    Returns
    -------
        tuple of 8 32-bit integers  - updated hash values
    

    """
    #step 1
    #the message schedule W (64 32-bit words) shs page 22
    #made from the first 16 words of the block and extended to 64 words
    W = []
    
    #First 16 words from the block (convert 4 bytes to 32-bit integer)
    for i in range(16):
        #append each 32 bit word to W in order of most significant byte first
        #append bytes starting from index i*4 to (i+1)*4 i.e block[0:4], block[4:8], ... 
        W.append(int.from_bytes(block[i*4:(i+1)*4], byteorder='big'))
    
    #Extend to 64 words using the schedule formula on page 22
    for i in range(16, 64):
        #get the next word by getting the word 2 postions before, 7 positions before, 15 positions before and 16 positions before
        #apply the sigma functions to the 2 and 15 position before words and sum all masking to keep only the lower 32 bits
        W.append((Sigma1_2(W[i-2]) + W[i-7] + Sigma0_2(W[i-15]) + W[i-16]) & 0xFFFFFFFF)
    
    #Step 2
    #Initialize the eight working variables with the (i-1)th hash value
    a, b, c, d, e, f, g, h = current
    
    #Step 3 page 23
    for t in range(64):
        #
        T1 = (h + Sigma1(e) + Ch(e, f, g) + sha256_constants[t] + W[t]) & 0xFFFFFFFF
        T2 = (Sigma0(a) + Maj(a, b, c)) & 0xFFFFFFFF
        
        h = g
        g = f
        f = e
        e = (d + T1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (T1 + T2) & 0xFFFFFFFF
    
    #Step 4 page 23
    #Compute the i-th intermediate hash value H(i) by adding working variables to current hash
    H0 = (current[0] + a) & 0xFFFFFFFF
    H1 = (current[1] + b) & 0xFFFFFFFF
    H2 = (current[2] + c) & 0xFFFFFFFF
    H3 = (current[3] + d) & 0xFFFFFFFF
    H4 = (current[4] + e) & 0xFFFFFFFF
    H5 = (current[5] + f) & 0xFFFFFFFF
    H6 = (current[6] + g) & 0xFFFFFFFF
    H7 = (current[7] + h) & 0xFFFFFFFF
    
    return (H0, H1, H2, H3, H4, H5, H6, H7)

## Problem 5: Passwords

In [20]:
#TODO 
# Problem 1
# Mark down explantions
# Referencing - secure hashing standard , 
# Problem 2
# call all methods in into a single callable function 
# test with various inputs
# referemce - secure hashing standard,geeks for geeks
# context added in markdown cells in the notebook
#problem 3
#Add tests
# references – secure hashing standard,realpython.com,geeks for geeks byte order,python docs
# markdown cells in the notebook
# problem 4
#comments explaining each step
# references – secure hashing standard
#markdown cells in the notebook
#test with various inputs

#Readme

